In [1]:
import numpy as np
import pandas as pd
import models.LAY2 as LAY
from DB.DB import db
from models import utils
from models.utils import metric_r, metric_c
import models.meta_models as meta_models

In [2]:
# 加载数据集
data_loader = db('DataBase/DB.csv')
# data_loader.get_data()
data_loader.data = data_loader.data.iloc[:20, :]
# 五折切割
train, test = next(data_loader.get_folds())

data already finished size =9022
fold0


In [3]:
train

,id,smiles,LogLD
0,100-00-5,C1=CC(=CC=C1[N+](=O)[O-])Cl,2.719213
1,100-02-7,C1=CC(=CC=C1[N+](=O)[O-])O,2.285557
2,100-14-1,C1=CC(=CC=C1CCl)[N+](=O)[O-],3.257439
3,100-15-2,CNC1=CC=C(C=C1)[N+](=O)[O-],3.301030
5,100-18-5,CC(C)C1=CC=C(C=C1)C(C)C,3.505150
6,100-20-9,C1=CC(=CC=C1C(=O)Cl)C(=O)Cl,3.397940
7,100-21-0,C1=CC(=CC=C1C(=O)O)C(=O)O,3.978089
8,100-28-7,C1=CC(=CC=C1N=C=O)[N+](=O)[O-],3.204120
9,100-29-8,CCOC1=CC=C(C=C1)[N+](=O)[O-],3.518514
11,100-36-7,CCN(CC)CCN,3.451786


train.to_csv("train.csv")
test.to_csv("test.csv")

In [11]:
# 定义模型结构
model_structure = utils.model_sequential(
    # meta_models.meta_RF(ECFP_Params=[2048,2]),
    # meta_models.meta_RF(ECFP_Params=[4096,2]),
    meta_models.meta_MPNN(),
    # meta_models.meta_AFP(),
    # meta_models.meta_DNN(ECFP_Params=[2048,2]),
    # meta_models.meta_GCN(),
    # meta_models.meta_SVR(),
    # meta_models.meta_GAT(),
    
)
model_structure.show_models()

The structure of Stacking model:
id= 0,Message passing neural network for molecular property prediction.


In [12]:
eb_model = LAY.ensemble_models_new(model_save_files='model_checkpoint_debug')
eb_model.train_meta_models(train,test,model_squential=model_structure,save=True)
eb_model.L1_test_df

Command line


The structure of Stacking model:
id= 0,Message passing neural network for molecular property prediction.
start training MPNN 


python e:\ProgramFiles\anaconda3\Lib\site-packages\ipykernel_launcher.py --ip=127.0.0.1 --stdin=9003 --control=9001 --hb=9000 --Session.signature_scheme="hmac-sha256" --Session.key=b"93ba2342-c883-41aa-8bc1-533e82595517" --shell=9002 --transport="tcp" --iopub=9004 --f=c:\Users\donkey\AppData\Roaming\jupyter\runtime\kernel-v2-285280e7Hji5rRUtk.json
Args
{'activation': 'ReLU',
 'adding_bond_types': True,
 'adding_h': False,
 'aggregation': 'mean',
 'aggregation_norm': 100,
 'atom_constraints': [],
 'atom_descriptor_scaling': True,
 'atom_descriptors': None,
 'atom_descriptors_path': None,
 'atom_descriptors_size': 0,
 'atom_features_size': 0,
 'atom_messages': False,
 'atom_targets': [],
 'batch_size': 50,
 'bias': False,
 'bias_solvent': False,
 'bond_constraints': [],
 'bond_descriptor_scaling': True,
 'bond_descriptors': None,
 'bond_descriptors_path': None,
 'bond_descriptors_size': 0,
 'bond_features_size': 0,
 'bond_targets': [],
 'cache_cutoff': 10000,
 'checkpoint_dir': None,
 'c

StopIteration: 

In [6]:
# 定义模型第二层结构
model_structure = utils.model_sequential(
    meta_models.L2_SVR(),
    meta_models.L2_RF(),
    meta_models.L2_RF(),
    meta_models.L2_SVR()
)
model_structure.show_models()

The structure of Stacking model:
id= 0,Support vector regression for the second layer of Stacking_model
id= 0,Random Forest Regressor for the second layer of Stacking_model
id= 0,Random Forest Regressor for the second layer of Stacking_model
id= 0,Support vector regression for the second layer of Stacking_model


In [7]:
eb_model.L2_training(model_structure,train,save=True)

The structure of Stacking model:
id= 0,Support vector regression for the second layer of Stacking_model
id= 0,Random Forest Regressor for the second layer of Stacking_model
id= 0,Random Forest Regressor for the second layer of Stacking_model
id= 0,Support vector regression for the second layer of Stacking_model
Start fitting SVR in L2
model_checkpoint_debug
Start fitting RF in L2
model_checkpoint_debug
Start fitting RF in L2
model_checkpoint_debug
Start fitting SVR in L2
model_checkpoint_debug


In [12]:
eb_model.L3_training(save =True)

In [13]:
utils.metric_r(eb_model.L1_train_df.true,eb_model.L1_train_df.meta_RF_0)

[0.2910663130957067, 0.3858978012652377, 0.0036627814500043154]

In [15]:
eb_model.L1_train_df

,true,meta_RF_0,meta_RF_1,meta_MPNN_2,meta_DNN_3,meta_SVR_4
0,2.719213,3.198336,3.196042,3.193726,3.113785,2.819399
1,2.285557,3.193409,3.190830,3.185762,3.100618,2.585268
2,3.257439,3.198336,3.196042,3.204944,3.093489,3.157237
3,3.301030,3.198336,3.196042,3.191794,3.107282,3.201258
4,3.505150,3.198336,3.196042,3.225988,3.378521,3.404799
5,3.397940,3.198336,3.196042,3.220609,3.305257,3.470244
6,3.978089,3.198336,3.196042,3.198808,3.287786,3.877719
7,3.204120,3.198336,3.196042,3.182270,3.115529,3.104353
8,3.518514,3.198336,3.196042,3.191297,3.126511,3.418050
9,3.451786,3.198336,3.196042,3.189742,3.078187,3.351718


In [30]:
utils.cal_df(eb_model.L3_train_df)

,nnls
MAE,0.055319
RMSE,0.072575
R2,0.964760


In [27]:
df = utils.cal_df(eb_model.L1_train_df)
dic =dict(df)
dic

{'meta_RF_0': MAE     0.291066
 RMSE    0.385898
 R2      0.003663
 Name: meta_RF_0, dtype: float64,
 'meta_RF_1': MAE     0.291622
 RMSE    0.385886
 R2      0.003722
 Name: meta_RF_1, dtype: float64,
 'meta_MPNN_2': MAE     0.289816
 RMSE    0.385902
 R2      0.003639
 Name: meta_MPNN_2, dtype: float64,
 'meta_DNN_3': MAE     0.279569
 RMSE    0.358746
 R2      0.138936
 Name: meta_DNN_3, dtype: float64,
 'meta_SVR_4': MAE     0.110836
 RMSE    0.121276
 R2      0.901596
 Name: meta_SVR_4, dtype: float64}

In [29]:
type(dic['meta_RF_0'])

pandas.core.series.Series

In [3]:
eb_model2 = LAY.ensemble_models_new(model_save_files='model_checkpoint_debug')
eb_model2.load_pretrained_metamodels(train)
utils.cal_df(eb_model2.L1_predict_df)
eb_model2.load_pretrained_L2models()
eb_model2.load_pretrained_L3model()

The structure of Stacking model:
id= 0,Random Forest Regressor useing ECFP
id= 1,Random Forest Regressor useing ECFP
id= 2,Message passing neural network for molecular property prediction.
id= 3,Deep neural network useing ECFP
id= 4,Support vector regression useing ECFP 
RF predict
0
RF predict
1
MPNN predict
Loading training args
Setting molecule featurization parameters to default.
Loading data


16it [00:00, 16151.35it/s]
100%|██████████| 16/16 [00:00<00:00, 16008.79it/s]


Validating SMILES
Test size = 16


  0%|          | 0/1 [00:00<?, ?it/s]

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "readout.1.weight".
Loading pretrained parameter "readout.1.bias".
Loading pretrained parameter "readout.4.weight".
Loading pretrained parameter "readout.4.bias".


100%|██████████| 1/1 [00:09<00:00,  9.80s/it]

Saving predictions to model_checkpoint_debug/MPNN/predict_df2.csv
Elapsed time = 0:00:10
DNN predict
SVR predict
0.2859692634999481
0.1149974278683569
0.2859692634999481
0.0
0.2859692634999481
0.8834727254026574
0.2859692634999481
0.0
use L3_predict_array to get output


In [4]:
eb_model2.L3_predict_array

array([3.85029193, 3.60420448, 4.37097971, 4.38373897, 4.64708994,
       4.61262836, 4.983377  , 4.30413479, 4.62359113, 4.58491044,
       4.30708835, 4.58947148, 4.19117002, 4.36917942, 4.32612616,
       3.86180519])

In [8]:
model = getattr(meta_models,f'meta_RF')

In [11]:
a = model()